<a href="https://colab.research.google.com/github/DataLake-FYP-Project/Object-Detection-Tracking/blob/KAN-16-Implement-object-count/vehicle/object-counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Apr 10 05:51:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install gdown

In [3]:
! rm -rf video.mp4

In [33]:
import gdown

# Update with your file's specific ID
file_id = "1NNhyO_PUrfy3dVat_BmlYaBSDCLIzo2Q"
url = f"https://drive.google.com/uc?id={file_id}"

output = "vehicle-counting.mp4"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1r0AVu38_1k2lA8unOhE0OYVDivQvbaVu
To: /content/tr.mp4
100%|██████████| 7.04M/7.04M [00:00<00:00, 68.4MB/s]


'tr.mp4'

In [22]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [34]:
SOURCE_VIDEO_PATH = "/kaggle/working/vehicle-counting.mp4"

In [7]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.0/112.6 GB disk)


In [8]:
!pip install supervision==0.3.0

from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.3.0


In [9]:
# settings
MODEL = "yolov8x.pt"

In [10]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 187MB/s]


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


## Track and count

In [35]:
import supervision as sv
from ultralytics import YOLO
import os
import json
import subprocess
import cv2
import numpy as np
from sklearn.cluster import KMeans
import webcolors
#import ffmpeg
from datetime import datetime, timedelta

# Function to check if a point is inside a polygon
def is_in_target_polygon(center_x, center_y, polygon):
    point = (center_x, center_y)
    return cv2.pointPolygonTest(np.array(polygon, dtype=np.int32), point, False) >= 0

# Perspective transformation: SOURCE and TARGET
SOURCE = np.array([
    [1252, 787],
    [2298, 803],
    [5039, 2159],
    [-550, 2159]
])

TARGET_WIDTH = 25
TARGET_HEIGHT = 250

# Constants for speed calculation
SCALE_FACTOR = 0.05
VEHICLE_POSITIONS = {}
FPS = 30

TARGET = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

# Compute perspective transformation matrix
perspective_transform = cv2.getPerspectiveTransform(SOURCE.astype(np.float32), TARGET.astype(np.float32))


TARGET_VIDEO_PATH = 'output_video.mp4'
FRAME_SAVE_DIR = 'frames/'
FRAME_DATA_PATH = 'frame_data.json'

# Initialize box annotator for drawing bounding boxes
box_annotator = sv.BoxAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2
)

# Open video info and frame generator
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH)

frame_width = video_info.width
left_lane_end = frame_width // 3
right_lane_start = 2 * (frame_width // 3)
id_counter = 1
id_map = {}
frame_data_list = []
vehicle_positions = {}
vehicle_times = {}
red_violators=set()
id_map,id_ctr={},1


os.makedirs(FRAME_SAVE_DIR, exist_ok=True)

line_y_position = video_info.height // 2

vehicle_crossings = {'entered': 0, 'exited': 0}
crossing_tracker = {}

color_dict = {
    "red": (255, 0, 0),
    "green": (0, 255, 0),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "white": (255, 255, 255),
    "black": (0, 0, 0),
    "gray": (128, 128, 128),
    "brown": (139, 69, 19),
    "pink": (255, 192, 203),
    "violet": (238, 130, 238),
    "light_red": (255, 102, 102),  # Added more shades
    "light_green": (102, 255, 102),
    "light_blue": (102, 102, 255),
    "light_yellow": (255, 255, 102),
}

# Red light violation detection
def detect_crossing_box(frame):
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)
    edges=cv2.Canny(blur,50,150)
    cnts,_=cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    boxes=[cv2.boundingRect(c) for c in cnts if cv2.boundingRect(c)[2]>cv2.boundingRect(c)[3]*3]
    if not boxes:
        return (200,430),(600,460)
    x,y,w,h = max(boxes,key=lambda b:b[2])
    return (x,y),(x+w,y+h)

def detect_light_color(frame,box):
    x1,y1,x2,y2=map(int,box)
    roi=frame[y1:y2,x1:x2]
    if roi.size==0: return "unknown"
    hsv=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    r1=cv2.inRange(hsv,(0,70,50),(10,255,255))
    r2=cv2.inRange(hsv,(160,70,50),(180,255,255))
    g=cv2.inRange(hsv,(40,40,40),(80,255,255))
    y=cv2.inRange(hsv,(20,100,100),(30,255,255))
    rc, gc, yc = cv2.countNonZero(r1|r2), cv2.countNonZero(g), cv2.countNonZero(y)
    if rc>gc and rc>yc and rc>50: return "red"
    if gc>rc and gc>yc and gc>50: return "green"
    if yc>rc and yc>gc and yc>50: return "yellow"
    return "unknown"

# Compute crossing box once
cap_tmp=cv2.VideoCapture(SOURCE_VIDEO_PATH)
_,first=cap_tmp.read(); cap_tmp.release()
crossing_box = detect_crossing_box(first)


def get_video_creation_time(video_path):
    cmd = [
        'ffprobe', '-v', 'error',
        '-show_entries', 'format_tags=creation_time',
        '-of', 'json', video_path
    ]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = json.loads(result.stdout)

    if 'creation_time' in output['format']['tags']:
        video_start_time_str = output['format']['tags']['creation_time']
        video_start_time = datetime.strptime(video_start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        return video_start_time
    else:
        return None

def datetime_converter(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError("Type not serializable")

# Function to get the closest color name based on RGB values
def closest_color(rgb):
    min_colors = {}
    for name, color in color_dict.items():
        distance = np.linalg.norm(np.array(color) - np.array(rgb))
        min_colors[name] = distance
    return min(min_colors, key=min_colors.get)

def get_exact_vehicle_color(bbox, frame, k=1):
    x1, y1, x2, y2 = map(int, bbox)
    vehicle_roi = frame[y1:y2, x1:x2]
    pixels = vehicle_roi.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_.astype(int)[0]
    dominant_color_rgb = (dominant_color[2], dominant_color[1], dominant_color[0])  # BGR to RGB
    closest_color_name = closest_color(dominant_color_rgb)

    return closest_color_name

# Function to calculate speed based on vehicle positions
def calculate_speed(center_x, center_y, tracker_id, frame_number, scale_factor=SCALE_FACTOR, fps=FPS):
    if tracker_id not in VEHICLE_POSITIONS:
        VEHICLE_POSITIONS[tracker_id] = (center_x, center_y, frame_number)

    prev_x, prev_y, prev_frame = VEHICLE_POSITIONS[tracker_id]
    displacement = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)
    time_elapsed = (frame_number - prev_frame) / fps
    speed = (displacement / time_elapsed) * scale_factor if time_elapsed > 0 else 0
    VEHICLE_POSITIONS[tracker_id] = (center_x, center_y, frame_number)

    return speed

def is_stopped(speed):
    return speed == 0

def get_vehicle_direction(center_x, center_y, tracker_id, vehicle_positions):
    direction = "Unknown"
    if tracker_id in vehicle_positions:
        prev_x, prev_y = vehicle_positions[tracker_id]
        if center_y > prev_y:
            direction = "Down"
        elif center_y < prev_y:
            direction = "Up"

    return direction

def get_lane(center_x, left_lane_end, right_lane_start):
    if center_x < left_lane_end:
        return "Left Lane"
    elif center_x > right_lane_start:
        return "Right Lane"
    else:
        return "Middle Lane"


def calculate_congestion_level(detections):
    return len(detections)

def handle_tracker_ids(tracker_ids, id_map, id_counter):
    updated_ids = []
    for tracker_id in tracker_ids:
        if tracker_id not in id_map:
            id_map[tracker_id] = id_counter
            id_counter += 1
        updated_ids.append(id_map[tracker_id])
    return updated_ids, id_map, id_counter



# Open output video stream
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(
            source=SOURCE_VIDEO_PATH,
            tracker='bytetrack.yaml',
            show=False,
            stream=True,
            agnostic_nms=True,
            persist=True
        )
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)



        creation_time = get_video_creation_time(SOURCE_VIDEO_PATH)

        if creation_time:
            VIDEO_START_TIME = creation_time
        else:
            VIDEO_START_TIME = datetime.now()

        if result.boxes.id is not None:

            updated_tracker_ids, id_map, id_counter = handle_tracker_ids(result.boxes.id.cpu().numpy().astype(int), id_map, id_counter)
            detections.tracker_id = updated_tracker_ids

                # Traffic light detection
            light="unknown"
            if result.boxes.id is not None:
                for box,cls in zip(result.boxes.xyxy,result.boxes.cls):
                    if model.model.names[int(cls)]=='traffic light':
                        light=detect_light_color(frame,box)
                        cv2.rectangle(frame,tuple(map(int,box[:2])),tuple(map(int,box[2:])),
                                      (0,0,255) if light=='red' else (0,255,0),2)
                        break

            # Remap tracker IDs
            if result.boxes.id is not None:
                tracker_ids=result.boxes.id.cpu().numpy().astype(int); upd=[]
                for t in tracker_ids:
                    if t not in id_map: id_map[t]=id_ctr; id_ctr+=1
                    upd.append(id_map[t])
                detections.tracker_id=upd

        for bbox, confidence, class_id, tracker_id in detections:
            if tracker_id is None:
                continue
            tracker_id = int(tracker_id)
            bbox = [float(coord) for coord in bbox]
            center_x = (bbox[0] + bbox[2]) / 2
            center_y = (bbox[1] + bbox[3]) / 2


            if tracker_id not in crossing_tracker:
                crossing_tracker[tracker_id] = {'crossed': False, 'last_position': bbox[1]}
            vehicle_color = get_exact_vehicle_color(bbox, frame)

            direction = get_vehicle_direction(center_x, center_y, tracker_id, vehicle_positions)
            lane = get_lane(center_x, left_lane_end, right_lane_start)

            vehicle_positions[tracker_id] = (center_x, center_y)
            if tracker_id not in crossing_tracker:
                crossing_tracker[tracker_id] = {'crossed': False, 'last_position': center_y}

            if not crossing_tracker[tracker_id]['crossed']:
                if center_y > line_y_position and crossing_tracker[tracker_id]['last_position'] <= line_y_position:
                    vehicle_crossings['entered'] += 1
                    crossing_tracker[tracker_id]['crossed'] = True  # Mark as crossed
                elif center_y < line_y_position and crossing_tracker[tracker_id]['last_position'] >= line_y_position:
                    vehicle_crossings['exited'] += 1
                    crossing_tracker[tracker_id]['crossed'] = True  # Mark as crossed

            crossing_tracker[tracker_id]['last_position'] = center_y
            cv2.line(frame, (0, line_y_position), (video_info.width, line_y_position), (0, 255, 0), 2)
            cv2.putText(frame, f"Entered: {vehicle_crossings['entered']}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(frame, f"Exited: {vehicle_crossings['exited']}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            congestion_level = calculate_congestion_level(detections)
            speed = calculate_speed(center_x, center_y, tracker_id, frame_number)
            stopped = bool(is_stopped(speed))

            # --- NEW VIOLATION: ENTERING the blue box while red ---
            inside = (crossing_box[0][0] <= center_x <= crossing_box[1][0]
                      and crossing_box[0][1] <= center_y <= crossing_box[1][1])
            violation = False
            if inside and light=='red' and tracker_id not in red_violators:
                violation = True
                red_violators.add(tracker_id)
            crossing_tracker[tracker_id]['inside'] = inside

            if is_in_target_polygon(center_x, center_y, SOURCE):
                if tracker_id not in vehicle_times:
                    vehicle_times[tracker_id] = {"entry": frame_number, "exit": frame_number}
                else:
                    vehicle_times[tracker_id]["exit"] = frame_number
                label = f"ID: {tracker_id} | {model.model.names[class_id]} {confidence:0.2f}| Speed: {speed:.2f} km/h"
                frame = box_annotator.annotate(
                            scene=frame,
                            detections=sv.Detections(
                                    xyxy=np.array([bbox]),  # Convert to numpy array
                                    confidence=np.array([confidence]),  # Convert to numpy array
                                    class_id=np.array([class_id]),  # Convert to numpy array
                                    tracker_id=np.array([tracker_id]) if tracker_id is not None else None  # Tracker ID optional
                            ),
                labels=[label]
                )

                frame_data = {
                            "frame_number": frame_number,
                            "congestion_level": congestion_level,  # Add congestion level
                            "traffic_light":light,
                            "detections": [
                                {
                                    "tracker_id": int(tracker_id),
                                    "class_id": int(class_id),
                                    "class_name": model.names[class_id],
                                    "direction": direction,
                                    "lane": lane,
                                    "vehicle_color": vehicle_color,
                                    "stopped": stopped,
                                    "confidence": float(confidence),
                                    "bbox": [float(coord) for coord in bbox],
                                    "Speed_kmh":speed,
                                    "Red light violation":violation,
                                    "entry_time": (
                    (VIDEO_START_TIME + timedelta(seconds=vehicle_times.get(int(tracker_id), {}).get("entry", 0) / FPS))
                    .strftime("%Y-%m-%d %H:%M:%S")
                    if vehicle_times.get(int(tracker_id)) else None
                ),
                "exit_time": (
                    (VIDEO_START_TIME + timedelta(seconds=vehicle_times.get(int(tracker_id), {}).get("exit", 0) / FPS))
                    .strftime("%Y-%m-%d %H:%M:%S")
                    if vehicle_times.get(int(tracker_id)) else None
                )
                                }
                                for bbox, confidence, class_id, tracker_id in detections
                            ]
                        }
                frame_data_list.append(frame_data)

        cv2.polylines(frame, [SOURCE.astype(np.int32)], isClosed=True, color=(0, 255, 0), thickness=2)
        warped_frame = cv2.warpPerspective(frame, perspective_transform, (TARGET_WIDTH, TARGET_HEIGHT))
        cv2.putText(frame,f"Light:{light.upper()}",(30,150),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)
        # Blue crossing box
        cv2.rectangle(frame,crossing_box[0],crossing_box[1],(255,0,0),2)
        cv2.imwrite(f"warped_frame_{frame_number:04d}.jpg", warped_frame)
        frame_path = os.path.join(FRAME_SAVE_DIR, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        sink.write_frame(frame)

    print(f"Vehicle tracking completed. Entered: {vehicle_crossings['entered']}, Exited: {vehicle_crossings['exited']}")


# Save frame data to a JSON file
with open(FRAME_DATA_PATH, 'w') as json_file:
    json.dump(frame_data_list, json_file, indent=4)

print(f"Frames saved to '{FRAME_SAVE_DIR}' and frame data saved to '{FRAME_DATA_PATH}'.")


video 1/1 (frame 1/453) /content/tr.mp4: 384x640 1 car, 7 traffic lights, 61.9ms
video 1/1 (frame 2/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.7ms
video 1/1 (frame 3/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 4/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 5/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 6/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 7/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 8/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 9/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 10/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 11/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 12/453) /content/tr.mp4: 384x640 1 car, 1 traffic light, 61.8ms
video 1/1 (frame 13/453) /content/t

### Line Crossing Violation

In [ ]:
import supervision as sv
from ultralytics import YOLO
import os
import json
import subprocess
import cv2
import numpy as np
from sklearn.cluster import KMeans
import webcolors
#import ffmpeg
from datetime import datetime, timedelta

# Function to check if a point is inside a polygon
def is_in_target_polygon(center_x, center_y, polygon):
    point = (center_x, center_y)
    return cv2.pointPolygonTest(np.array(polygon, dtype=np.int32), point, False) >= 0

# Perspective transformation: SOURCE and TARGET
SOURCE = np.array([
    [1252, 787],
    [2298, 803],
    [5039, 2159],
    [-550, 2159]
])

LINE_POINTS = np.array([
    [2298, 803],
    [5039, 2159]
])
x1, y1 = LINE_POINTS[0]
x2, y2 = LINE_POINTS[1]

def get_y_on_line(x, x1, y1, x2, y2):
    if x2 == x1:
        return float('inf')  # avoid division by zero
    slope = (y2 - y1) / (x2 - x1)
    return y1 + slope * (x - x1)


TARGET_WIDTH = 25
TARGET_HEIGHT = 250

# Constants for speed calculation
SCALE_FACTOR = 0.05
VEHICLE_POSITIONS = {}
FPS = 30

TARGET = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

# Compute perspective transformation matrix
perspective_transform = cv2.getPerspectiveTransform(SOURCE.astype(np.float32), TARGET.astype(np.float32))


TARGET_VIDEO_PATH = 'output_video.mp4'
FRAME_SAVE_DIR = 'frames/'
FRAME_DATA_PATH = 'frame_data.json'

# Initialize box annotator for drawing bounding boxes
box_annotator = sv.BoxAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2
)

# Open video info and frame generator
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.video.get_video_frames_generator(SOURCE_VIDEO_PATH)

frame_width = video_info.width
left_lane_end = frame_width // 3
right_lane_start = 2 * (frame_width // 3)
id_counter = 1
id_map = {}
frame_data_list = []
vehicle_positions = {}
vehicle_times = {}
red_violators=set()
id_map,id_ctr={},1


os.makedirs(FRAME_SAVE_DIR, exist_ok=True)

line_y_position = video_info.height // 2

vehicle_crossings = {'entered': 0, 'exited': 0}
crossing_tracker = {}

color_dict = {
    "red": (255, 0, 0),
    "green": (0, 255, 0),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "white": (255, 255, 255),
    "black": (0, 0, 0),
    "gray": (128, 128, 128),
    "brown": (139, 69, 19),
    "pink": (255, 192, 203),
    "violet": (238, 130, 238),
    "light_red": (255, 102, 102),  # Added more shades
    "light_green": (102, 255, 102),
    "light_blue": (102, 102, 255),
    "light_yellow": (255, 255, 102),
}

# Red light violation detection
def detect_crossing_box(frame):
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)
    edges=cv2.Canny(blur,50,150)
    cnts,_=cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    boxes=[cv2.boundingRect(c) for c in cnts if cv2.boundingRect(c)[2]>cv2.boundingRect(c)[3]*3]
    if not boxes:
        return (200,430),(600,460)
    x,y,w,h = max(boxes,key=lambda b:b[2])
    return (x,y),(x+w,y+h)

def detect_light_color(frame,box):
    x1,y1,x2,y2=map(int,box)
    roi=frame[y1:y2,x1:x2]
    if roi.size==0: return "unknown"
    hsv=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    r1=cv2.inRange(hsv,(0,70,50),(10,255,255))
    r2=cv2.inRange(hsv,(160,70,50),(180,255,255))
    g=cv2.inRange(hsv,(40,40,40),(80,255,255))
    y=cv2.inRange(hsv,(20,100,100),(30,255,255))
    rc, gc, yc = cv2.countNonZero(r1|r2), cv2.countNonZero(g), cv2.countNonZero(y)
    if rc>gc and rc>yc and rc>50: return "red"
    if gc>rc and gc>yc and gc>50: return "green"
    if yc>rc and yc>gc and yc>50: return "yellow"
    return "unknown"

# Compute crossing box once
cap_tmp=cv2.VideoCapture(SOURCE_VIDEO_PATH)
_,first=cap_tmp.read(); cap_tmp.release()
crossing_box = detect_crossing_box(first)


def get_video_creation_time(video_path):
    cmd = [
        'ffprobe', '-v', 'error',
        '-show_entries', 'format_tags=creation_time',
        '-of', 'json', video_path
    ]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = json.loads(result.stdout)

    if 'creation_time' in output['format']['tags']:
        video_start_time_str = output['format']['tags']['creation_time']
        video_start_time = datetime.strptime(video_start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        return video_start_time
    else:
        return None

def datetime_converter(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError("Type not serializable")

# Function to get the closest color name based on RGB values
def closest_color(rgb):
    min_colors = {}
    for name, color in color_dict.items():
        distance = np.linalg.norm(np.array(color) - np.array(rgb))
        min_colors[name] = distance
    return min(min_colors, key=min_colors.get)

def get_exact_vehicle_color(bbox, frame, k=1):
    x1, y1, x2, y2 = map(int, bbox)
    vehicle_roi = frame[y1:y2, x1:x2]
    pixels = vehicle_roi.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_.astype(int)[0]
    dominant_color_rgb = (dominant_color[2], dominant_color[1], dominant_color[0])  # BGR to RGB
    closest_color_name = closest_color(dominant_color_rgb)

    return closest_color_name

# Function to calculate speed based on vehicle positions
def calculate_speed(center_x, center_y, tracker_id, frame_number, scale_factor=SCALE_FACTOR, fps=FPS):
    if tracker_id not in VEHICLE_POSITIONS:
        VEHICLE_POSITIONS[tracker_id] = (center_x, center_y, frame_number)

    prev_x, prev_y, prev_frame = VEHICLE_POSITIONS[tracker_id]
    displacement = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)
    time_elapsed = (frame_number - prev_frame) / fps
    speed = (displacement / time_elapsed) * scale_factor if time_elapsed > 0 else 0
    VEHICLE_POSITIONS[tracker_id] = (center_x, center_y, frame_number)

    return speed

def is_stopped(speed):
    return speed == 0

def get_vehicle_direction(center_x, center_y, tracker_id, vehicle_positions):
    direction = "Unknown"
    if tracker_id in vehicle_positions:
        prev_x, prev_y = vehicle_positions[tracker_id]
        if center_y > prev_y:
            direction = "Down"
        elif center_y < prev_y:
            direction = "Up"

    return direction

def get_lane(center_x, left_lane_end, right_lane_start):
    if center_x < left_lane_end:
        return "Left Lane"
    elif center_x > right_lane_start:
        return "Right Lane"
    else:
        return "Middle Lane"


def calculate_congestion_level(detections):
    return len(detections)

def handle_tracker_ids(tracker_ids, id_map, id_counter):
    updated_ids = []
    for tracker_id in tracker_ids:
        if tracker_id not in id_map:
            id_map[tracker_id] = id_counter
            id_counter += 1
        updated_ids.append(id_map[tracker_id])
    return updated_ids, id_map, id_counter



# Open output video stream
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(
            source=SOURCE_VIDEO_PATH,
            tracker='bytetrack.yaml',
            show=False,
            stream=True,
            agnostic_nms=True,
            persist=True
        )
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)



        creation_time = get_video_creation_time(SOURCE_VIDEO_PATH)

        if creation_time:
            VIDEO_START_TIME = creation_time
        else:
            VIDEO_START_TIME = datetime.now()

        if result.boxes.id is not None:

            updated_tracker_ids, id_map, id_counter = handle_tracker_ids(result.boxes.id.cpu().numpy().astype(int), id_map, id_counter)
            detections.tracker_id = updated_tracker_ids

                # Traffic light detection
            light="unknown"
            if result.boxes.id is not None:
                for box,cls in zip(result.boxes.xyxy,result.boxes.cls):
                    if model.model.names[int(cls)]=='traffic light':
                        light=detect_light_color(frame,box)
                        cv2.rectangle(frame,tuple(map(int,box[:2])),tuple(map(int,box[2:])),
                                      (0,0,255) if light=='red' else (0,255,0),2)
                        break

            # Remap tracker IDs
            if result.boxes.id is not None:
                tracker_ids=result.boxes.id.cpu().numpy().astype(int); upd=[]
                for t in tracker_ids:
                    if t not in id_map: id_map[t]=id_ctr; id_ctr+=1
                    upd.append(id_map[t])
                detections.tracker_id=upd

        for bbox, confidence, class_id, tracker_id in detections:
            if tracker_id is None:
                continue
            tracker_id = int(tracker_id)
            bbox = [float(coord) for coord in bbox]
            center_x = (bbox[0] + bbox[2]) / 2
            center_y = (bbox[1] + bbox[3]) / 2


            if tracker_id not in crossing_tracker:
                crossing_tracker[tracker_id] = {
                    'crossed': False,
                    'last_position': bbox[1],
                    'last_position_new': bbox[1]  # ✅ for new line
                }
            
            vehicle_color = get_exact_vehicle_color(bbox, frame)
            
            direction = get_vehicle_direction(center_x, center_y, tracker_id, vehicle_positions)
            lane = get_lane(center_x, left_lane_end, right_lane_start)
            
            vehicle_positions[tracker_id] = (center_x, center_y)
            
            # ✅ Update last_position_new if not already in crossing_tracker (for safety)
            if 'last_position_new' not in crossing_tracker[tracker_id]:
                crossing_tracker[tracker_id]['last_position_new'] = center_y
            
            # 🚥 Original line crossing logic
            if not crossing_tracker[tracker_id]['crossed']:
                if center_y > line_y_position and crossing_tracker[tracker_id]['last_position'] <= line_y_position:
                    vehicle_crossings['entered'] += 1
                    crossing_tracker[tracker_id]['crossed'] = True  # Mark as crossed
                elif center_y < line_y_position and crossing_tracker[tracker_id]['last_position'] >= line_y_position:
                    vehicle_crossings['exited'] += 1
                    crossing_tracker[tracker_id]['crossed'] = True  # Mark as crossed
            
            line_y_at_center = get_y_on_line(center_x, x1, y1, x2, y2)
            line_crossing = False
            if (center_y > line_y_at_center and crossing_tracker[tracker_id]['last_position_new'] <= line_y_at_center) or \
               (center_y < line_y_at_center and crossing_tracker[tracker_id]['last_position_new'] >= line_y_at_center):
                line_crossing = True
            
            # ✅ Update last positions for next frame
            crossing_tracker[tracker_id]['last_position'] = center_y
            crossing_tracker[tracker_id]['last_position_new'] = center_y

            cv2.line(frame, (0, line_y_position), (video_info.width, line_y_position), (0, 255, 0), 2)
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"Entered: {vehicle_crossings['entered']}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(frame, f"Exited: {vehicle_crossings['exited']}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            congestion_level = calculate_congestion_level(detections)
            speed = calculate_speed(center_x, center_y, tracker_id, frame_number)
            stopped = bool(is_stopped(speed))

            # --- NEW VIOLATION: ENTERING the blue box while red ---
            inside = (crossing_box[0][0] <= center_x <= crossing_box[1][0]
                      and crossing_box[0][1] <= center_y <= crossing_box[1][1])
            violation = False
            if inside and light=='red' and tracker_id not in red_violators:
                violation = True
                red_violators.add(tracker_id)
            crossing_tracker[tracker_id]['inside'] = inside

            if is_in_target_polygon(center_x, center_y, SOURCE):
                if tracker_id not in vehicle_times:
                    vehicle_times[tracker_id] = {"entry": frame_number, "exit": frame_number}
                else:
                    vehicle_times[tracker_id]["exit"] = frame_number
                label = f"ID: {tracker_id} | {model.model.names[class_id]} {confidence:0.2f}| Speed: {speed:.2f} km/h"
                frame = box_annotator.annotate(
                            scene=frame,
                            detections=sv.Detections(
                                    xyxy=np.array([bbox]),  # Convert to numpy array
                                    confidence=np.array([confidence]),  # Convert to numpy array
                                    class_id=np.array([class_id]),  # Convert to numpy array
                                    tracker_id=np.array([tracker_id]) if tracker_id is not None else None  # Tracker ID optional
                            ),
                labels=[label]
                )

                frame_data = {
                            "frame_number": frame_number,
                            "congestion_level": congestion_level,  # Add congestion level
                            "traffic_light":light,
                            "detections": [
                                {
                                    "tracker_id": int(tracker_id),
                                    "class_id": int(class_id),
                                    "class_name": model.names[class_id],
                                    "direction": direction,
                                    "lane": lane,
                                    "vehicle_color": vehicle_color,
                                    "stopped": stopped,
                                    "confidence": float(confidence),
                                    "bbox": [float(coord) for coord in bbox],
                                    "Speed_kmh":speed,
                                    "Red light violation":violation,
                                    "line crossing": line_crossing,
                                    "entry_time": (
                    (VIDEO_START_TIME + timedelta(seconds=vehicle_times.get(int(tracker_id), {}).get("entry", 0) / FPS))
                    .strftime("%Y-%m-%d %H:%M:%S")
                    if vehicle_times.get(int(tracker_id)) else None
                ),
                "exit_time": (
                    (VIDEO_START_TIME + timedelta(seconds=vehicle_times.get(int(tracker_id), {}).get("exit", 0) / FPS))
                    .strftime("%Y-%m-%d %H:%M:%S")
                    if vehicle_times.get(int(tracker_id)) else None
                )
                                }
                                for bbox, confidence, class_id, tracker_id in detections
                            ]
                        }
                frame_data_list.append(frame_data)

        cv2.polylines(frame, [SOURCE.astype(np.int32)], isClosed=True, color=(0, 255, 0), thickness=2)
        warped_frame = cv2.warpPerspective(frame, perspective_transform, (TARGET_WIDTH, TARGET_HEIGHT))
        cv2.putText(frame,f"Light:{light.upper()}",(30,150),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)
        # Blue crossing box
        cv2.rectangle(frame,crossing_box[0],crossing_box[1],(255,0,0),2)
        cv2.imwrite(f"warped_frame_{frame_number:04d}.jpg", warped_frame)
        frame_path = os.path.join(FRAME_SAVE_DIR, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        sink.write_frame(frame)

    print(f"Vehicle tracking completed. Entered: {vehicle_crossings['entered']}, Exited: {vehicle_crossings['exited']}")


# Save frame data to a JSON file
with open(FRAME_DATA_PATH, 'w') as json_file:
    json.dump(frame_data_list, json_file, indent=4)

print(f"Frames saved to '{FRAME_SAVE_DIR}' and frame data saved to '{FRAME_DATA_PATH}'.")

In [36]:
from IPython.display import FileLink
FileLink(r'output_video.mp4')

/content/output_video.mp4